In [ ]:
"""Incremental Principal Components Analysis."""

# Author: Kyle Kastner <kastnerkyle@gmail.com>
#         Giorgio Patrini
# License: BSD 3 clause

import numpy as np
from scipy import linalg

from .base import _BasePCA
from ..utils import check_array, gen_batches
from ..utils.extmath import svd_flip, _incremental_mean_and_var


class IncrementalKernelPCA(_BasePCA):
    """Incremental principal components analysis (IPCA).

    Linear dimensionality reduction using Singular Value Decomposition of
    the data, keeping only the most significant singular vectors to
    project the data to a lower dimensional space. The input data is centered
    but not scaled for each feature before applying the SVD.

    Depending on the size of the input data, this algorithm can be much more
    memory efficient than a PCA.

    This algorithm has constant memory complexity, on the order
    of ``batch_size``, enabling use of np.memmap files without loading the
    entire file into memory.

    The computational overhead of each SVD is
    ``O(batch_size * n_features ** 2)``, but only 2 * batch_size samples
    remain in memory at a time. There will be ``n_samples / batch_size`` SVD
    computations to get the principal components, versus 1 large SVD of
    complexity ``O(n_samples * n_features ** 2)`` for PCA.

    Read more in the :ref:`User Guide <IncrementalPCA>`.

    Parameters
    ----------
    n_components : int or None, (default=None)
        Number of components to keep. If ``n_components `` is ``None``,
        then ``n_components`` is set to ``min(n_samples, n_features)``.

    whiten : bool, optional
        When True (False by default) the ``components_`` vectors are divided
        by ``n_samples`` times ``components_`` to ensure uncorrelated outputs
        with unit component-wise variances.

        Whitening will remove some information from the transformed signal
        (the relative variance scales of the components) but can sometimes
        improve the predictive accuracy of the downstream estimators by
        making data respect some hard-wired assumptions.

    copy : bool, (default=True)
        If False, X will be overwritten. ``copy=False`` can be used to
        save memory but is unsafe for general use.

    batch_size : int or None, (default=None)
        The number of samples to use for each batch. Only used when calling
        ``fit``. If ``batch_size`` is ``None``, then ``batch_size``
        is inferred from the data and set to ``5 * n_features``, to provide a
        balance between approximation accuracy and memory consumption.

    Attributes
    ----------
    components_ : array, shape (n_components, n_features)
        Components with maximum variance.

    explained_variance_ : array, shape (n_components,)
        Variance explained by each of the selected components.

    explained_variance_ratio_ : array, shape (n_components,)
        Percentage of variance explained by each of the selected components.
        If all components are stored, the sum of explained variances is equal
        to 1.0.

    singular_values_ : array, shape (n_components,)
        The singular values corresponding to each of the selected components.
        The singular values are equal to the 2-norms of the ``n_components``
        variables in the lower-dimensional space.

    mean_ : array, shape (n_features,)
        Per-feature empirical mean, aggregate over calls to ``partial_fit``.

    var_ : array, shape (n_features,)
        Per-feature empirical variance, aggregate over calls to
        ``partial_fit``.

    noise_variance_ : float
        The estimated noise covariance following the Probabilistic PCA model
        from Tipping and Bishop 1999. See "Pattern Recognition and
        Machine Learning" by C. Bishop, 12.2.1 p. 574 or
        http://www.miketipping.com/papers/met-mppca.pdf.

    n_components_ : int
        The estimated number of components. Relevant when
        ``n_components=None``.

    n_samples_seen_ : int
        The number of samples processed by the estimator. Will be reset on
        new calls to fit, but increments across ``partial_fit`` calls.

    Examples
    --------
    >>> from sklearn.datasets import load_digits
    >>> from sklearn.decomposition import IncrementalPCA
    >>> X, _ = load_digits(return_X_y=True)
    >>> transformer = IncrementalPCA(n_components=7, batch_size=200)
    >>> # either partially fit on smaller batches of data
    >>> transformer.partial_fit(X[:100, :])
    IncrementalPCA(batch_size=200, copy=True, n_components=7, whiten=False)
    >>> # or let the fit function itself divide the data into batches
    >>> X_transformed = transformer.fit_transform(X)
    >>> X_transformed.shape
    (1797, 7)

    Notes
    -----
    Implements the incremental PCA model from:
    *D. Ross, J. Lim, R. Lin, M. Yang, Incremental Learning for Robust Visual
    Tracking, International Journal of Computer Vision, Volume 77, Issue 1-3,
    pp. 125-141, May 2008.*
    See https://www.cs.toronto.edu/~dross/ivt/RossLimLinYang_ijcv.pdf

    This model is an extension of the Sequential Karhunen-Loeve Transform from:
    *A. Levy and M. Lindenbaum, Sequential Karhunen-Loeve Basis Extraction and
    its Application to Images, IEEE Transactions on Image Processing, Volume 9,
    Number 8, pp. 1371-1374, August 2000.*
    See https://www.cs.technion.ac.il/~mic/doc/skl-ip.pdf

    We have specifically abstained from an optimization used by authors of both
    papers, a QR decomposition used in specific situations to reduce the
    algorithmic complexity of the SVD. The source for this technique is
    *Matrix Computations, Third Edition, G. Holub and C. Van Loan, Chapter 5,
    section 5.4.4, pp 252-253.*. This technique has been omitted because it is
    advantageous only when decomposing a matrix with ``n_samples`` (rows)
    >= 5/3 * ``n_features`` (columns), and hurts the readability of the
    implemented algorithm. This would be a good opportunity for future
    optimization, if it is deemed necessary.

    References
    ----------
    D. Ross, J. Lim, R. Lin, M. Yang. Incremental Learning for Robust Visual
    Tracking, International Journal of Computer Vision, Volume 77,
    Issue 1-3, pp. 125-141, May 2008.

    G. Golub and C. Van Loan. Matrix Computations, Third Edition, Chapter 5,
    Section 5.4.4, pp. 252-253.

    See also
    --------
    PCA
    KernelPCA
    SparsePCA
    TruncatedSVD
    """

    def __init__(self, n_components=None, whiten=False, copy=True,
                 batch_size=None, m0, mmax=None, kernel=kernel_error, adjust=False,
                     nystrom=False, maxiter=500, n):
        self.n_components = n_components
        self.whiten = whiten
        self.copy = copy
        self.batch_size = batch_size

    def fit(self, X, y=None):
        """Fit the model with X, using minibatches of size batch_size.

        Parameters
        ----------
        X : array-like, shape (n_samples, n_features)
            Training data, where n_samples is the number of samples and
            n_features is the number of features.

        y : Ignored

        Returns
        -------
        self : object
            Returns the instance itself.
        """
        self.components_ = None
        self.n_samples_seen_ = 0
        self.mean_ = .0
        self.var_ = .0
        self.singular_values_ = None
        self.explained_variance_ = None
        self.explained_variance_ratio_ = None
        self.singular_values_ = None
        self.noise_variance_ = None

        X = check_array(X, copy=self.copy, dtype=[np.float64, np.float32])
        n_samples, n_features = X.shape

        if self.batch_size is None:
            self.batch_size_ = 5 * n_features
        else:
            self.batch_size_ = self.batch_size

        for batch in gen_batches(n_samples, self.batch_size_,
                                 min_batch_size=self.n_components or 0):
            self.partial_fit(X[batch], check_input=False)

        return self

    def partial_fit(self, X, y=None, check_input=True):
        """Incremental fit with X. All of X is processed as a single batch.

        Parameters
        ----------
        X : array-like, shape (n_samples, n_features)
            Training data, where n_samples is the number of samples and
            n_features is the number of features.
        check_input : bool
            Run check_array on X.

        y : Ignored

        Returns
        -------
        self : object
            Returns the instance itself.
        """
        if check_input:
            X = check_array(X, copy=self.copy, dtype=[np.float64, np.float32])
        n_samples, n_features = X.shape
        if not hasattr(self, 'components_'):
            self.components_ = None

        if self.n_components is None:
            if self.components_ is None:
                self.n_components_ = min(n_samples, n_features)
            else:
                self.n_components_ = self.components_.shape[0]
        elif not 1 <= self.n_components <= n_features:
            raise ValueError("n_components=%r invalid for n_features=%d, need "
                             "more rows than columns for IncrementalPCA "
                             "processing" % (self.n_components, n_features))
        elif not self.n_components <= n_samples:
            raise ValueError("n_components=%r must be less or equal to "
                             "the batch number of samples "
                             "%d." % (self.n_components, n_samples))
        else:
            self.n_components_ = self.n_components

        if (self.components_ is not None) and (self.components_.shape[0] !=
                                               self.n_components_):
            raise ValueError("Number of input features has changed from %i "
                             "to %i between calls to partial_fit! Try "
                             "setting n_components to a fixed value." %
                             (self.components_.shape[0], self.n_components_))

        # This is the first partial_fit
        if not hasattr(self, 'n_samples_seen_'):
            self.n_samples_seen_ = 0
            self.mean_ = .0
            self.var_ = .0

        # Update stats - they are 0 if this is the fisrt step
        col_mean, col_var, n_total_samples = \
            _incremental_mean_and_var(
                X, last_mean=self.mean_, last_variance=self.var_,
                last_sample_count=np.repeat(self.n_samples_seen_, X.shape[1]))
        n_total_samples = n_total_samples[0]

        # Whitening
        if self.n_samples_seen_ == 0:
            # If it is the first step, simply whiten X
            X -= col_mean
        else:
            col_batch_mean = np.mean(X, axis=0)
            X -= col_batch_mean
            # Build matrix of combined previous basis and new data
            mean_correction = \
                np.sqrt((self.n_samples_seen_ * n_samples) /
                        n_total_samples) * (self.mean_ - col_batch_mean)
            X = np.vstack((self.singular_values_.reshape((-1, 1)) *
                          self.components_, X, mean_correction))

        U, S, V = linalg.svd(X, full_matrices=False)
        U, V = svd_flip(U, V, u_based_decision=False)
        explained_variance = S ** 2 / (n_total_samples - 1)
        explained_variance_ratio = S ** 2 / np.sum(col_var * n_total_samples)

        self.n_samples_seen_ = n_total_samples
        self.components_ = V[:self.n_components_]
        self.singular_values_ = S[:self.n_components_]
        self.mean_ = col_mean
        self.var_ = col_var
        self.explained_variance_ = explained_variance[:self.n_components_]
        self.explained_variance_ratio_ = \
            explained_variance_ratio[:self.n_components_]
        if self.n_components_ < n_features:
            self.noise_variance_ = \
                explained_variance[self.n_components_:].mean()
        else:
            self.noise_variance_ = 0.
        return self

In [ ]:
image_folder = 'E:\\TMT_DATA_2\\T2-Armazones'
txt = r'E:\\TMT_DATA_2\\T2-Armazones\\usability_T2-Armazones_200510_200802_COMBINED .txt'
#def get_data_array(root_dir,txt):
b = read_txt(txt,image_folder)
b.remove(b[0])
b.remove(b[-1])
txtlist = b
str0 =  ''.join(b[0][0:3])
dir0 = image_folder + "\\" + str0
print(dir0)
hour =  ''.join(b[0][3])
n = len(txtlist)
batch_n = 50
iter_n = n//batch_n + 1
transformer = IncrementalPCA(n_components=7, batch_size=batch_n)
C1_transformed = np.zeros(1)
C1_transformed = C1_transformed.tolist()
C2_transformed = np.zeros(1)
C2_transformed = C2_transformed.tolist()
R1_transformed = np.zeros(1)
R1_transformed = R1_transformed.tolist()
R2_transformed = np.zeros(1)
R2_transformed = R2_transformed.tolist()
label_out = np.zeros(n)
data1C_new = np.zeros([batch_n,np.shape(Loc_C)[0]])
data2C_new = np.zeros([batch_n,np.shape(Loc_C)[0]])
data1R_new = np.zeros([batch_n,np.shape(Loc_R)[0]])
data2R_new = np.zeros([batch_n,np.shape(Loc_R)[0]])
total = 0
for i in range (iter_n):
    print("training on ",i+1," of ",iter_n," batches")
    data1C_new = data1C_new - data1C_new
    data2C_new = data2C_new - data2C_new
    data1R_new = data1R_new - data1R_new
    data2R_new = data2R_new - data2R_new
    for j in range(batch_n):
        str0 =  ''.join(txtlist[i*batch_n+j][0:3])
        dir0 = image_folder + "\\" + str0
        hour =  ''.join(txtlist[i*batch_n+j][3])
        label = ''.join(txtlist[i*batch_n+j][4])
        label = convert_label(label)
        data1_sum, data2_sum = get_hourly_mean(dir0,hour)
        if np.sum(data1_sum) == 0:
            continue
        else:
            label_out[i*batch_n+j] = label
            data1C_new[j,:] = Get_Data(data1_sum,Loc_C).T
            total = total + 1
            data2C_new[j,:] = Get_Data(data2_sum,Loc_C).T
            data1R_new[j,:] = Get_Data(data1_sum,Loc_R).T
            data2R_new[j,:] = Get_Data(data2_sum,Loc_R).T
    
    C1_transformed.extend(transformer.fit_transform(data1C_new))
    R1_transformed.extend(transformer.fit_transform(data1R_new))
    C2_transformed.extend(transformer.fit_transform(data2C_new))
    R2_transformed.extend(transformer.fit_transform(data2R_new))
    
    
C1_transformed = C1_transformed[1:]
R1_transformed = R1_transformed[1:]
R1_transformed = R1_transformed[1:]
R2_transformed = R2_transformed[1:]

    

In [22]:
import numpy as np
a = [[1,2,3],[4,5,6]]
b = np.random.permutation(100)
print(a[1],a[0])

[4, 5, 6] [1, 2, 3]


In [23]:
import matplotlib.pyplot as plt
from astropy.visualization import astropy_mpl_style
from astropy.io import fits
import astropy
plt.style.use(astropy_mpl_style)
from astropy.utils.data import get_pkg_data_filename
import numpy as np
#import ogr
#import shapely.wkt
#import shapely.geometry
import urllib.request
import zipfile
import json
import csv
from collections import Counter
import itertools
from collections import defaultdict
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import csv
import os, glob
from sklearn import decomposition
from sklearn import cluster
from sklearn import preprocessing
from sklearn.datasets import load_digits
from sklearn.decomposition import IncrementalPCA
from scipy import spatial
from io import StringIO
import importlib
from mpl_toolkits.mplot3d import Axes3D
from astropy.utils.data import get_pkg_data_filename
from astropy.io import fits
import PIL
from PIL import Image
import scipy.misc
import math
from scipy.spatial.distance import pdist, squareform

In [ ]:
def data_normalization(data): #normalizes data to a gaussian
    new_data = preprocessing.normalize(np.log(data))
    return new_data

def Get_Round_Pixel_Locations(data,radius,center_x,center_y): 
    #gets the coordinates of pixels in the round arrea. Outputs a n*2 array
    len_y = np.shape(data)[0]
    len_x = np.shape(data)[1]
    out = [[None,None]] #stores the coordinate of the pixels
    for j in range(0,len_y-1):
        for i in range(0,len_x-1):
            if ((i-center_x)**2 + (j-center_y)**2) < radius**2:
                out.append([i,j])
    out = np.delete(out, 0, 0)  # delete first row of out
    return out #Returns the Loc: location of the pixels

def Get_Ring_Pixel_Locations(data,radius_inner,radius_outer,center_x,center_y): 
    #gets the coordinates of pixels in the round arrea. Outputs a n*2 array
    len_y = np.shape(data)[0]
    len_x = np.shape(data)[1]
    out = [[None,None]] #stores the coordinate of the pixels
    for j in range(0,len_y-1):
        for i in range(0,len_x-1):
            if ((((i-center_x)**2 + (j-center_y)**2) <= radius_outer**2) & (((i-center_x)**2 + (j-center_y)**2) >= radius_inner**2) ):
                out.append([i,j])
    out = np.delete(out, 0, 0)  # delete first row of out
    return out #Returns the Loc: location of the pixels

def Get_Data(data, Loc): 
    #returns the converted pixels using the coordinates
    n = np.shape(Loc)[0]
    out = np.zeros([np.shape(Loc)[0],1])
    for i in range(0,n-1):
        out[i] = data[Loc[i,0],Loc[i,1]]
    return out

def Reconstruct_Data(data_Vec,Loc,x_len,y_len):
    n = np.shape(Loc)[0]
    out = np.zeros([x_len,y_len])
    for i in range(0,n-1):
        out[Loc[i,0],Loc[i,1]] = data_Vec[i]
    return out

def read_txt(txt,filename): #reads the txt file, converts the txt file to list
    a = []
    with open(txt, 'r') as f:
        while True:
            line = f.readline() 
            b = line.split()
            a.append(b)
            if not line:
                return a
                break     
                
def convert_label(label): #tokenize the labels
    if label == 'n':  
        out = 0
    elif label == 'm':
        out = 1
    elif label == 'o':
        out = 2
    elif label == 'p':
        out = 3
    elif label == 'i':
        out = 4
    elif label == 'u':
        out = 5
    else:
        out = 6
    return out

# def get_hourly_mean(rootDir,hour,Loc_C,Loc_R): #calculates the mean data hourly, outputs the blue and red band data
#     list_dirs = os.walk(rootDir)
#     a = ''
#     cnt1 = 0
#     cnt2 = 0
#     data1C_sum = np.zeros([np.shape(Loc_C)[0]])
#     data2C_sum = np.zeros([np.shape(Loc_C)[0]])
#     data1R_sum = np.zeros([np.shape(Loc_R)[0]])
#     data2R_sum = np.zeros([np.shape(Loc_R)[0]])
#     for root, dirs, files in list_dirs: 
#         for d in files: 
#             path = os.path.join(root, d)
#             str0 = path.split('\\')[-1]
#             str1 = str0[3]
#             str2 = str0[14]+str0[15]
            
#             if str1 == "b":
#                 if str2 == hour:
#                     cnt1 += 1
#                     image_file = path
#                     fits.info(image_file)
#                     image_data = fits.getdata(image_file, ext=0)
#                     image_data = data_normalization(np.log(image_data))
#                     data_C1 = Get_Data(image_data,Loc_C)
#                     #data_R1 = Get_Data(image_data,Loc_R)
#                     data1C_sum = data1C_sum + data_C1
#                     #data1R_sum = data1R_sum + data_R1
                    
#             elif str1 == "r":
#                 if str2 == hour:
#                     cnt2 += 1
#                     image_file = path
#                     #fits.info(image_file)
#                     image_data = fits.getdata(image_file, ext=0)
#                     image_data = data_normalization(np.log(image_data))
#                     #data_C2 = Get_Data(image_data,Loc_C)
#                     #data_R2 = Get_Data(image_data,Loc_R)
#                     #data2C_sum = data2C_sum + data_C2
#                     #data2R_sum = data2R_sum + data_R2
#             #print(str)
#     data1C_sum = data1C_sum/cnt1
#     data1R_sum = data1R_sum/cnt1
#     data2C_sum = data2C_sum/cnt2
#     data2R_sum = data2R_sum/cnt2
#     return data1C_sum, data1R_sum, data2C_sum,data2R_sum
def get_hourly_mean(rootDir,hour): #calculates the mean data hourly, outputs the blue and red band data
    list_dirs = os.walk(rootDir)
    a = ''
    cnt1 = 0
    cnt2 = 0
    data1 = np.zeros([1024,1024])
    data2 = np.zeros([1024,1024])
    for root, dirs, files in list_dirs: 
        for d in files: 
            path = os.path.join(root, d)
            str0 = path.split('\\')[-1]
            str1 = str0[3]
            str2 = str0[14]+str0[15]
            
            if str1 == "b":
                if str2 == hour:
                    cnt1 += 1
                    image_file = path
                    #fits.info(image_file)
                    
                    image_data = fits.getdata(image_file, ext=0)
                    #image_data = data_normalization(np.log(image_data))
                    if image_data.shape[0] != 1024:
                        cnt1 -=1
                        continue
                    else:
                        data1 = data1 + image_data
                    #print(np.max(data1))
                    
            elif str1 == "r":
                if str2 == hour:
                    cnt2 += 1
                    image_file = path
                    #fits.info(image_file)
                    
                    image_data = fits.getdata(image_file, ext=0)
                    
                    #image_data = data_normalization(np.log(image_data))
                    if image_data.shape[0] != 1024:
                        cnt2 -= 1
                        continue
                    else:
                        data2 = data2 + image_data
            #print(str)
    if cnt1 * cnt2 == 0:
        data1 = np.zeros([1024,1024])
        data2 = np.zeros([1024,1024])
        return data1,data2
    data1 = data1/cnt1
    data2 = data2/cnt2
    data1 = data_normalization(data1)
    data2 = data_normalization(data2)
    return data1, data2    

def data_organizer(txtlist,Loc_C,Loc_R): #reshapes the data and restructure all data in the list. data: d*n label: n*1
    num_data = len(txtlist)
    #print(num_data)
    data_bC = np.zeros([np.shape(Loc_C)[0],num_data])
    #print(np.shape(data_bC))
#     data_bR = np.zeros([np.shape(Loc_R)[0],num_data])
#     data_rC = np.zeros([np.shape(Loc_C)[0],num_data])
#     data_rR = np.zeros([np.shape(Loc_R)[0],num_data])
    label_out = np.zeros(num_data)
    for i in range (num_data):
        str0 =  ''.join(txtlist[i][0:3])
        dir0 = image_folder + "\\" + str0
        hour =  ''.join(txtlist[i][3])
        label = ''.join(txtlist[i][4])
        label = convert_label(label)
        label_out[i] = label
        data1C_sum, data1R_sum, data2C_sum,data2R_sum = get_hourly_mean(dir0,hour,Loc_C,Loc_R)
        data_bC[:,i] = data1C_sum
        return data_bC,label_out
#         data_bR[:,i] = data1R_sum
#         data_rC[:,i] = data2C_sum
#         data_rR[:,i] = data2R_sum
#     return data_bC,data_bR,data_rC,data_rR,label_out
#def get_next_data(txtlist,Loc_C,Loc_R,i):
    
    

In [ ]:
#generate a toy data
#image_file = get_pkg_data_filename('tutorials/FITS-images/HorseHead.fits')
image_file = r'C:\USB backup\Sem1, 2019\ENGN8602\sample_data\to_b20061127ut005945s19380.fits.gz'
#image_file = r'E:\T1-Tolar.20061126.asca\net\data\ASCA\20061126\to_b20061127ut000753s16260.fits.gz'
print(image_file)
fits.info(image_file)
image_data = fits.getdata(image_file, ext=0)
plt.figure()
#plt.imshow(data_normalization(np.log(image_data)), cmap='gray')
#plt.colorbar()
image_data = data_normalization(np.log(image_data))
r_out = 470
r_in = 470/np.sqrt(2)
x = 55+470
y = 35+470
#get locations
Loc_C = Get_Round_Pixel_Locations(image_data,r_in,x,y)
Loc_R = Get_Ring_Pixel_Locations(image_data,r_in,r_out,x,y)
#get data
#data_C = Get_Data(image_data,Loc_C)
#data_R = Get_Data(image_data,Loc_R)

print(np.shape(Loc_C)[0])
print(np.shape(Loc_R)[0])

In [ ]:
image_folder = 'E:\\TMT_DATA_2\\T2-Armazones'
txt = r'E:\\TMT_DATA_2\\T2-Armazones\\usability_T2-Armazones_200510_200802_COMBINED .txt'
#def get_data_array(root_dir,txt):
b = read_txt(txt,image_folder)
b.remove(b[0])
b.remove(b[-1])
txtlist = b
str0 =  ''.join(b[0][0:3])
dir0 = image_folder + "\\" + str0
print(dir0)
hour =  ''.join(b[0][3])
n = len(txtlist)
batch_n = 50
iter_n = n//batch_n + 1
transformer = IncrementalPCA(n_components=7, batch_size=batch_n)
C1_transformed = np.zeros(1)
C1_transformed = C1_transformed.tolist()
C2_transformed = np.zeros(1)
C2_transformed = C2_transformed.tolist()
R1_transformed = np.zeros(1)
R1_transformed = R1_transformed.tolist()
R2_transformed = np.zeros(1)
R2_transformed = R2_transformed.tolist()
label_out = np.zeros(n)
data1C_new = np.zeros([batch_n,np.shape(Loc_C)[0]])
data2C_new = np.zeros([batch_n,np.shape(Loc_C)[0]])
data1R_new = np.zeros([batch_n,np.shape(Loc_R)[0]])
data2R_new = np.zeros([batch_n,np.shape(Loc_R)[0]])
total = 0
for i in range (iter_n):
    print("training on ",i+1," of ",iter_n," batches")
    data1C_new = data1C_new - data1C_new
    data2C_new = data2C_new - data2C_new
    data1R_new = data1R_new - data1R_new
    data2R_new = data2R_new - data2R_new
    for j in range(batch_n):
        str0 =  ''.join(txtlist[i*batch_n+j][0:3])
        dir0 = image_folder + "\\" + str0
        hour =  ''.join(txtlist[i*batch_n+j][3])
        label = ''.join(txtlist[i*batch_n+j][4])
        label = convert_label(label)
        data1_sum, data2_sum = get_hourly_mean(dir0,hour)
        if np.sum(data1_sum) == 0:
            continue
        else:
            label_out[i*batch_n+j] = label
            data1C_new[j,:] = Get_Data(data1_sum,Loc_C).T
            total = total + 1
            data2C_new[j,:] = Get_Data(data2_sum,Loc_C).T
            data1R_new[j,:] = Get_Data(data1_sum,Loc_R).T
            data2R_new[j,:] = Get_Data(data2_sum,Loc_R).T
    
    C1_transformed.extend(transformer.fit_transform(data1C_new))
    R1_transformed.extend(transformer.fit_transform(data1R_new))
    C2_transformed.extend(transformer.fit_transform(data2C_new))
    R2_transformed.extend(transformer.fit_transform(data2R_new))
    
    
C1_transformed = C1_transformed[1:]
R1_transformed = R1_transformed[1:]
R1_transformed = R1_transformed[1:]
R2_transformed = R2_transformed[1:]

    

In [26]:
class Counter:
    def __init__(self, low, high):
        self.current = low
        self.high = high

    def fit(self): # Python 3: def __next__(self)
        if self.current > self.high:
            raise StopIteration
        else:
            self.current += 1
            return (self.current - 1, self.high)

counter = Counter(3,8)
for i in range(5):
    print(counter.fit())

(3, 8)
(4, 8)
(5, 8)
(6, 8)
(7, 8)
